### stage dataset

extend stint dataset to k laps dataset

base: 18./make_stint_dataset

build dataset with feature extraction

+ remove first and final 'stint'
+ add neighbors information as features


# Imports

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Load Data

In [2]:
import os
os.getcwd()


'/scratch_hdd/hpda/indycar/notebook/18.FinalTest'

In [3]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
    
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [4]:
def load_data(event, year=0):
    #inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    if year>0:
        inputfile = '../data/final/C_'+ event +'-' + year + '.csv'
    else:
        inputfile = '../data/final/C_'+ event +'.csv'
    
    #outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [5]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

In [6]:
def draw_laptime(ax, ycol='LapTime'):
    
    ymax = np.max(y)
    
    #yellow flags
    for yflag in yellowflags:
        ax.bar(yflag[0],ymax, width = yflag[1] - yflag[0], align='edge', color = 'y', zorder=-100)

    #plt.plot(np.arange(laptime.shape[0]), laptime, marker='.')
    ax.scatter(x, y, marker='.')    

    #pit stops    
    ax.scatter(pits, y[pits-1], marker='x', color='r')
    for pit in pits:
        ax.bar(pit,height = y[pits-1], width = .8, align='center', color = 'r', zorder=-100)
    
    #ax.set_xlim((0, 33))
    #ax.set_ylim((0, 201))
    ax.set_xlabel('Lap')
    ax.set_ylabel(ycol)
    
def draw_pitstop(ax, pits):
    #pit stops    
    ax.scatter(pits, y[pits-1], marker='x', color='r')
    for pit in pits:
        ax.bar(pit,height = y[pits-1], width = .8, align='center', color = 'r', zorder=-100)

    
def draw_yellowflag(ax,yellowflags):
    ymax = np.max(y)
    #yellow flags
    for yflag in yellowflags:
        ax.bar(yflag[0],ymax, width = yflag[1] - yflag[0], align='edge', color = 'y', zorder=-100)

### bulid the dataset

In [7]:
def get_stagedata(TRIM=2, include_final=False):
    """
    input: acldata, rankdata, the whole dataset
    output: stagedata['car_number','stage','start_lap','end_lap','start_rank','end_rank']
    """
    carlist = set(acldata['car_number'])
    data = []
    for curcarno in carlist:
        car = acldata[acldata['car_number']==curcarno]
        #print(car['time_diff'].describe())

        cols=['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']
        colid={key:idx for idx, key in enumerate(cols)}

        cardata = rankdata[rankdata['car_number'] == curcarno]

        carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
            ['completed_laps','rank','car_number','lap_status','track_status',
            'pit_stop_count','current_status','start_position']].values]

        pits=[]
        yellowflags=[]
        lastflag = 'x'
        
        stage = 0
        start_lap = 0
        pit_lap = 0
        pit_in_caution = 0
        for row in carstatus:
            lap = int(row[colid['completed_laps']])
            
            if lap==0:
                #hex2int
                start_rank = int(row[colid['start_position']], 16)     
            
            if row[colid['lap_status']]=='P':
                pits.append(lap)
                end_lap = lap - TRIM
                end_rank = carstatus[end_lap][colid['rank']]
                #add one record
                #stagedata['car_number','stage','start_lap','end_lap','start_rank','end_rank']
                
                # predict k laps ahead, rather than the whole stint
                target_lap = start_lap + _predictlen
                if target_lap >= len(carstatus):
                    break
                    
                target_rank = carstatus[target_lap][colid['rank']]

                #data.append([curcarno,stage,start_lap, end_lap, pit_lap, pit_in_caution, start_rank, end_rank])
                data.append([curcarno,stage,start_lap, target_lap, pit_lap, pit_in_caution, start_rank, target_rank])
                
                # prepare for the next
                stage += 1
                start_lap = end_lap  # end_lap + 1?
                start_rank = end_rank
                pit_lap = lap  #current is pit lap
                pit_in_caution = 1 if row[colid['track_status']]=='Y' else 0
                

            if row[colid['track_status']]=='Y':
                if lastflag != 'Y':       
                    #start
                    yellowflags.append(lap)
            else:
                if lastflag == 'Y':       
                    #end
                    yellowflags.append(lap)        
            lastflag = row[colid['track_status']]

        if include_final:
            #final
            end_lap = carstatus[-1][colid['completed_laps']]
            end_rank = carstatus[-1][colid['rank']]
            #add one record
            #stagedata['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution,'start_rank','end_rank']
            #data.append([curcarno,stage,start_lap, end_lap, pit_lap, pit_in_caution, start_rank, end_rank])
            
            # predict k laps ahead, rather than the whole stint
            target_lap = start_lap + _predictlen
            if target_lap < len(carstatus):
                target_rank = carstatus[target_lap][colid['rank']]

                #data.append([curcarno,stage,start_lap, end_lap, pit_lap, pit_in_caution, start_rank, end_rank])
                data.append([curcarno,stage,start_lap, target_lap, pit_lap, pit_in_caution, start_rank, target_rank])
            
        
        #pit lap
        pits = np.array(pits)
        #start, end lap
        #
        yellowflags = np.array(yellowflags)
        if (yellowflags.shape[0] % 2)==1:
            print('crash?:carno=', curcarno)
            yellowflags = []
        else:
            yellowflags = np.array(yellowflags).reshape((-1,2))  
            
    data = np.array(data)
    #['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution,'start_rank','end_rank']
    df = pd.DataFrame({'car_number': data[:, 0], 'stage': data[:, 1],'start_lap': data[:, 2],
                      'end_lap': data[:, 3], 'pit_lap': data[:, 4], 'pit_in_caution': data[:, 5],
                       'start_rank': data[:, 6],'end_rank': data[:, 7]})
    return df, data

### events

In [8]:
def extract_features(stagedata):
    """
    input: (alldata, rankdata, acldata, flagdata, stage, data)
    
    target
    eventid
    car_number
    stageid
    
    #0, gobal info 
    firststage  ;  1/0    
    pit_in_caution; 1/0
    start_position  ; #
    
    # 0 order of #rank
    start_rank  ;  #rank
    start_rank_ratio; ; #rank/carnum 
    top_pack    ;  top5 1/0 
    bottom_pack  ; bottom5 1/0 
    
    average_rank  ; previous stage
    average_rank_all  ; all previous stages
    
    # 1 order of #rank
    change_in_rank; previous stage
    change_in_rank_all;  all previous stages
    # 2 order of #rank
    rate_of_change; previous stage
    rate_of_change_all;  all previous stages
    
    #neighbors;   
    prev_nb0_change_in_rank
    prev_nb1_change_in_rank
    prev_nb2_change_in_rank
    follow_nb0_change_in_rank
    follow_nb1_change_in_rank
    follow_nb2_change_in_rank
    
    # more features
    laptime_green_mean_prev  ; mean and std of the laptime in green laps of previous stage
    laptime_green_std_prev
    laptime_green_mean_all  ; mean and std of the laptime in green laps before
    laptime_green_std_all
    
    laptime_mean_prev  ; mean and std of the laptime in all laps of previous stage
    laptime_std_prev
    laptime_mean_all  ; mean and std of the laptime in all laps before
    laptime_std_all
    
    laps_prev   ; lap number of the previous stage
    laps_after_last_pitstop  ; lap number after the last pitstop
    
    pittime_prev   ; pittime of previous pitstop
    
    
    """
    eventid = 0
    cols=['target','start_lap',
             'eventid','car_number','stageid',
             'firststage','pit_in_caution','start_position',
             'start_rank','start_rank_ratio','top_pack','bottom_pack',
             'average_rank','average_rank_all',
             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all',
             'laptime_green_mean_prev','laptime_green_std_prev','laptime_green_mean_all','laptime_green_std_all', 
             'laptime_mean_prev','laptime_std_prev','laptime_mean_all','laptime_std_all', 
             'laps_prev','laps_after_last_pitstop','pittime_prev',     
             'prev_nb0_change_in_rank','prev_nb1_change_in_rank','prev_nb2_change_in_rank',
             'follow_nb0_change_in_rank','follow_nb1_change_in_rank','follow_nb2_change_in_rank']
    colid={key:idx for idx, key in enumerate(cols)}
    
    datacols = ['car_number','stage','start_lap','end_lap','pit_lap', 'pit_in_caution', 'start_rank','end_rank']
    datacolid={key:idx for idx, key in enumerate(datacols)}
    
    #maintain a <car,lap, (rank, stage)> index 
    idxcols = ['car_number','completed_laps','rank','stageid']
    carindex = {}
    
    stagedf = pd.DataFrame([],columns=cols)
    for event in stagedata.keys():
        alldata, rankdata, acldata, flagdata, stage, data = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)

        # init the index
        #carindex[eventid] = np.array((totalcars, totallaps, 2))
        carindex[eventid] = pd.DataFrame([],columns=idxcols)
        
        records = []
        
        laphist = []
        rankhist = []
        rankchghist = []        
        #['car_number','stage','start_lap','end_lap',pit_lap, pit_in_caution, 'start_rank','end_rank']
        for row in data:
            # a new record
            rec={}
            #status, 'start_lap', start a new car
            if row[datacolid['start_lap']] == 0:
                start_position = row[datacolid['start_rank']]
                laphist = []
                rankhist = []
                rankchghist = []
            
            #target 
            rec['target'] = row[datacolid['end_rank']] - row[datacolid['start_rank']]
            rec['start_lap'] = row[datacolid['start_lap']]
            
            #features
            rec['eventid'] = eventid
            rec['car_number'] = row[datacolid['car_number']]
            rec['stageid'] = row[datacolid['stage']]
            rec['firststage'] = 0 if row[datacolid['stage']]==0 else 1
            rec['pit_in_caution'] = row[datacolid['pit_in_caution']]
            rec['start_position'] = start_position
            rec['start_rank'] = row[datacolid['start_rank']]
            rec['start_rank_ratio'] = rec['start_rank'] / totalcars
            rec['top_pack'] = 1 if rec['start_rank']<=5 else 0
            #todo, not accurate here
            rec['bottom_pack'] = 1 if rec['start_rank']>=totalcars-5 else 0
            #get from laphist
            if laphist:
                #previous stage
                prev_stage = laphist[-1]
                rec['average_rank'] = np.mean(list(prev_stage['rank']))
                rec['average_rank_all'] = np.mean(rankhist)
            else:
                rec['average_rank'] = rec['start_rank']
                rec['average_rank_all'] = rec['start_rank']
                
            #change in rank
            if rankchghist:
                rec['change_in_rank'] = rankchghist[-1]
                rec['change_in_rank_all'] = np.mean(rankchghist)
            else:
                rec['change_in_rank'] = 0
                rec['change_in_rank_all'] = 0
                
            #rate of change in rank
            if len(rankchghist)>2:
                rec['rate_of_change'] = rankchghist[-1] - rankchghist[-2]
                rec['rate_of_change_all'] = (rankchghist[-1] - rankchghist[0])/(len(rankchghist)-1)
            else:
                rec['rate_of_change'] = 0
                rec['rate_of_change_all'] = 0            
                
            # init neighbor info as 0
            rec['prev_nb0_change_in_rank'] = 0
            rec['prev_nb1_change_in_rank'] = 0
            rec['prev_nb2_change_in_rank'] = 0
            rec['follow_nb0_change_in_rank'] = 0
            rec['follow_nb1_change_in_rank'] = 0
            rec['follow_nb2_change_in_rank'] = 0
            
            # laptime
            if not laphist:
                rec['laptime_green_mean_prev'] = 0
                rec['laptime_green_std_prev'] = 0
                rec['laptime_green_mean_all'] = 0
                rec['laptime_green_std_all'] = 0
                rec['laptime_mean_prev'] = 0
                rec['laptime_std_prev'] = 0
                rec['laptime_mean_all'] = 0
                rec['laptime_std_all'] = 0
                rec['laps_prev'] = 0
                rec['laps_after_last_pitstop'] = 0
                rec['pittime_prev'] = 0
            else:
                #previous stage greeds
                prev_stage = laphist[-1]
                green_laps = list(prev_stage[prev_stage['track_status']=='G']['last_laptime'])
                rec['laptime_green_mean_prev'] = np.mean(green_laps)
                rec['laptime_green_std_prev'] =  np.std(green_laps)
                all_laps = list(prev_stage['last_laptime'])
                rec['laptime_mean_prev'] = np.mean(all_laps)
                rec['laptime_std_prev'] =  np.std(all_laps)
                rec['laps_prev'] = len(all_laps)
                
                last_pit_lap = int(prev_stage['last_pitted_lap'].values[-1], 16) 
                rec['laps_after_last_pitstop'] = rec['start_lap'] - last_pit_lap
                
                #get two laps
                pit_data = rankdata[(rankdata['car_number']== rec['car_number']) & 
                               (rankdata['completed_laps']>=last_pit_lap) &
                               (rankdata['completed_laps']<last_pit_lap+2)
                              ]
                rec['pittime_prev'] = np.mean(list(pit_data['last_laptime']))
                
                # all previous laps laptime
                all_data = rankdata[(rankdata['car_number']== rec['car_number']) & 
                               (rankdata['completed_laps']<rec['start_lap'])
                              ]
                green_laps = list(all_data[all_data['track_status']=='G']['last_laptime'])
                rec['laptime_green_mean_all'] = np.mean(green_laps)
                rec['laptime_green_std_all'] =  np.std(green_laps)
                all_laps = list(all_data['last_laptime'])
                rec['laptime_mean_all'] = np.mean(all_laps)
                rec['laptime_std_all'] =  np.std(all_laps)                
            
            #add one record
            records.append([rec[col] for col in cols])
            
            #update for the new stage
            lapdata = rankdata[(rankdata['car_number']== rec['car_number']) & 
                               (rankdata['completed_laps']>=row[datacolid['start_lap']]) &
                               (rankdata['completed_laps']<row[datacolid['end_lap']])
                              ]
            
            # some pit stops are gapless
            if len(lapdata)==0:
                print('Caution:zero pits at carno:', rec['car_number'],
                     'start_lap:',row[datacolid['start_lap']],
                     'end_lap:',row[datacolid['end_lap']])
                continue
                
            laphist.append(lapdata)
            rankhist.extend(list(lapdata['rank']))
            rankchghist.append(row[datacolid['end_rank']] - row[datacolid['start_rank']])
            
            # add to index, go through the lapdata
            #or _lap in range(row[datacolid['start_lap']],row[datacolid['end_lap']]):
            for index, row in lapdata.iterrows():
                #carindex[eventid][rec['car_number'], row['completed_laps'], 0] = row['rank']
                #carindex[eventid][rec['car_number'], row['completed_laps'], 1 = rec['stageid']
                laprec = [rec['car_number'], row['completed_laps'], row['rank'],rec['stageid']]
                df = pd.DataFrame([laprec],columns=idxcols)
                carindex[eventid] = pd.concat([carindex[eventid], df], ignore_index=True)
            
          
        # push this event into stage dataframe
        print('eventid=%d, records=%d'%(eventid, len(records)))
        df = pd.DataFrame(records,columns=cols)
        stagedf = pd.concat([stagedf, df], ignore_index=True)
        
        #for the next new event
        eventid += 1
        
    return stagedf, carindex

def update_neighbor(stagedf, carindex):
    # go through stage data to add neighbor info
    # carindex[eventid] <car, lap, (rank, stage)
    ndf = stagedf.copy()
    
    #stagedf.index = pd.RangeIndex(len(stagedf.index))
        
    for index, row in stagedf.iterrows():
        
        eventid = row['eventid']
        
        currank = row['start_rank']
        curlap = row['start_lap']
        if curlap == 0:
            continue
        
        df = carindex[row['eventid']]
        df_lap = df[df['completed_laps']==curlap]
        prev0 = df_lap[df_lap['rank'] == currank - 1]
        prev1 = df_lap[df_lap['rank'] == currank - 2]
        prev2 = df_lap[df_lap['rank'] == currank - 3]
        follow0 = df_lap[df_lap['rank'] == currank + 1]
        follow1 = df_lap[df_lap['rank'] == currank + 2]
        follow2 = df_lap[df_lap['rank'] == currank + 3]
        
        recs = [prev0, prev1, prev2, follow0, follow1,follow2]
        recloc = ['prev_nb0_change_in_rank','prev_nb1_change_in_rank','prev_nb2_change_in_rank',
                 'follow_nb0_change_in_rank','follow_nb1_change_in_rank','follow_nb2_change_in_rank']
        for recidx, rec in enumerate(recs):
            if rec.empty:
                continue
                
            car_number = rec['car_number']
            stageid = rec['stageid']
            #print('event:%d, car:%d, stage:%d, index:%d'%(eventid, car_number, stageid, index))
            #find the record of prev0 by eventid, carno, stageid
            #neighbor = stagedf[(stagedf['eventid'] == eventid) & (stagedf['car_number'] == car_number) & (stagedf['stageid'] == stageid)]
            #neighbor = ndf[(ndf['eventid'] == eventid) & (ndf['car_number'] == car_number) & (ndf['stageid'] == stageid)]
            neighbor = ndf.query('eventid == %d & car_number==%d & stageid==%d'%(eventid,car_number,stageid))
            
            if neighbor.empty:
                print('error: empty neighbor at: '%(eventid,car_number,stageid))
                continue
            
            # update current row
            #print('index=%d, col=%s'%(index, recloc[recidx]))
            stagedf.loc[index,recloc[recidx]] = neighbor['change_in_rank'].values[0]
            #stagedf.loc[index,recloc[recidx]] = 1
                                
    return stagedf

### load data

In [9]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0
years = ['2013','2014','2015','2016','2017','2018','2019']
#events = ['Indy500']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}
dbid = f'Indy500_{years[0]}_{years[-1]}'

In [10]:
traindata = None
_trim = 0
_predictlen = 2
_include_final = False
include_str = '1' if _include_final else '0'
output_file = f'stage-indy500-2013-2019-end{include_str}-t{_trim}-l{_predictlen}.csv'

for event in events:
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event)
    
    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))
    
    stage, data = get_stagedata(TRIM=_trim, include_final = _include_final)
    stagedata[event] = (alldata, rankdata, acldata, flagdata, stage, data)
    
    if traindata is None:
        traindata = data
    else:
        traindata = np.vstack((traindata, data))

count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 5, 8, 9, 11, 12, 77, 14, 78, 18, 19, 20, 83, 22, 55, 25, 26}
#cars= 19
count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 41, 55, 60, 63, 77, 78, 81, 83, 91, 98}
#cars= 33
Indy500-2013: carno=33, lapnum=201
crash?:carno= 6


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {33, 2, 3, 34, 5, 68, 98, 8, 11, 12, 77, 14, 16, 17, 18, 21, 22, 25, 26, 28}
#cars= 20
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 41, 63, 67, 68, 77, 83, 91, 98}
#cars= 33
Indy500-2014: carno=33, lapnum=201
crash?:carno= 9
crash?:carno= 10
crash?:carno= 67
crash?:carno= 83
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 98, 5, 6, 7, 9, 11, 14, 15, 48, 83, 21, 22, 24, 26, 27, 28, 29}
#cars= 20
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 32, 41, 43, 48, 63, 83, 88, 98}
#cars= 33
Indy500-2

In [11]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [12]:
stagedf0, carindex = extract_features(stagedata)

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


eventid=0, records=198
eventid=1, records=228
eventid=2, records=198
eventid=3, records=259
eventid=4, records=248
eventid=5, records=192
eventid=6, records=216


In [13]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [14]:
stagedf0[(stagedf0['car_number']==12) & (stagedf0['eventid']==5)]

,target,start_lap,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
1171,-1,0,5,12,0,0,0,3,3,0.090909,...,0.000000,0,0,0.00000,0,0,0,0,0,0
1172,6,32,5,12,1,1,0,3,4,0.121212,...,8.527139,2,32,66.10705,0,0,0,0,0,0
1173,-2,50,5,12,2,1,1,3,6,0.181818,...,11.242669,2,18,61.85245,0,0,0,0,0,0
1174,8,94,5,12,3,1,0,3,1,0.030303,...,23.296174,2,44,117.01525,0,0,0,0,0,0
1175,5,129,5,12,4,1,0,3,2,0.060606,...,20.554273,2,35,59.24225,0,0,0,0,0,0


In [17]:
#stagedf0.reset_index(inplace=True)
#stagedf = update_neighbor(stagedf0, carindex) 
stagedf = stagedf0

In [18]:
%debug

> <ipython-input-8-8842916918f4>(279)update_neighbor()
    277             #neighbor = stagedf[(stagedf['eventid'] == eventid) & (stagedf['car_number'] == car_number) & (stagedf['stageid'] == stageid)]
    278             #neighbor = ndf[(ndf['eventid'] == eventid) & (ndf['car_number'] == car_number) & (ndf['stageid'] == stageid)]
--> 279             neighbor = ndf.query('eventid == %d & car_number==%d & stageid==%d'%(eventid,car_number,stageid))
    280 
    281             if neighbor.empty:

ipdb> quit


In [19]:
stagedf = stagedf.drop(columns=['start_lap'])

In [20]:
stagedf = stagedf[stagedf['firststage']!=0]
#suffix='withneighbor-newfeatures'
#stagedf.to_csv('stage-%s-%s-t%s.csv'%('indy500-2013-2019',suffix, _trim))
stagedf.to_csv(output_file )

In [ ]:
sys.exit(0)

### test

In [ ]:
stagedf[(stagedf['car_number']==12) & (stagedf['eventid']==5)]

In [ ]:
newfeatures=['laptime_green_mean_prev','laptime_green_std_prev','laptime_green_mean_all','laptime_green_std_all', 
 'laptime_mean_prev','laptime_std_prev','laptime_mean_all','laptime_std_all', 
 'laps_prev','laps_after_last_pitstop','pittime_prev'] 
            
stagedf[(stagedf['eventid']==1)][newfeatures].head(10)

In [ ]:
np.max(stagedf)

In [ ]:
stagedf[stagedf['pittime_prev']>=3644][newfeatures]

In [ ]:
chkdata = stagedata['Pocono'][1]

In [ ]:
chkdata[(chkdata['car_number']==27) &(chkdata['lap_status']=='P')]

In [ ]:
chkdata[(chkdata['car_number']==27) &(chkdata['completed_laps']<15)]

In [ ]:
#check the integrity of the data(laptime)
#for event in stagedata.keys():
    alldata, rankdata, acldata, flagdata, stage, data = stagedata[event]
    
chkdata = stagedata['Pocono'][2]
chkdata.head(10)

### verify pitstop dataset

In [ ]:
alldata, rankdata, acldata, flagdata,_,_ = stagedata['Indy500-2018']
rankdata[(rankdata['car_number']==12) & (rankdata['lap_status']=='P')]

In [ ]:
pitlaps = np.array([31,49,93,128])
rankdata[(rankdata['car_number']==12) & (rankdata['completed_laps'].isin(pitlaps-1))]

In [ ]:
rankdata[(rankdata['car_number']==12) & (rankdata['completed_laps']==127)]

### verify dataset(sync with ranknet)

In [ ]:
output_file

In [21]:
stagedf[(stagedf['eventid']==5)]

,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,top_pack,...,laptime_std_all,laps_prev,laps_after_last_pitstop,pittime_prev,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
1132,2,5,1,1,1,0,4,3,0.090909,1,...,8.421594,2,32,66.06825,0,0,0,0,0,0
1133,2,5,1,2,1,1,4,3,0.090909,1,...,11.374793,2,18,61.12735,0,0,0,0,0,0
1134,11,5,1,3,1,1,4,9,0.272727,0,...,23.928703,2,20,119.20330,0,0,0,0,0,0
1135,12,5,1,4,1,0,4,9,0.272727,0,...,23.131872,2,31,117.22760,0,0,0,0,0,0
1136,11,5,1,5,1,0,4,8,0.242424,0,...,20.608892,2,34,59.83245,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,2,5,88,7,1,0,22,17,0.515152,0,...,21.858578,2,30,104.37145,0,0,0,0,0,0
1319,6,5,98,1,1,0,12,7,0.212121,0,...,7.174138,2,32,63.28770,0,0,0,0,0,0
1320,3,5,98,2,1,1,12,10,0.303030,0,...,9.690737,2,18,61.57250,0,0,0,0,0,0
1321,8,5,98,3,1,0,12,15,0.454545,0,...,22.939522,2,42,116.28375,0,0,0,0,0,0


In [22]:
df = stagedf0[stagedf0['firststage']!=0]
test = df[(df['eventid']==5)][['car_number','start_lap','start_rank','target']].values
test.shape


(159, 4)

In [23]:
test[:,1] = test[:,1]-1
test[:,2] = test[:,2]-1
test[:,3] = test[:,2] + test[:,3]
dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])

In [24]:
dfout[dfout['carno']==12]

,carno,startlap,startrank,endrank
33,12,31,3,9
34,12,49,5,3
35,12,93,0,8
36,12,128,1,6


In [26]:
dfout.to_csv('test_db_ml_l2_2018.csv')

In [ ]:
max(15, np.nan)

In [ ]:
max([15, np.nan, 179,np.nan])